# Callbacks
- 학습 도중에 어떤 event를 일으킴
    - learning rate 줄일 수도 있고, tensor board에 recode를 할 수도 있고, check point 저장(모델에 weight 저장) 등등

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
os.listdir('dataset/cifar/')

['labels.txt', 'test', 'train']

In [6]:
train_paths = glob('dataset/cifar/train/*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob('dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()


# Callbacks

In [15]:
start_time = datetime.now()

In [16]:
start_time

datetime.datetime(2020, 7, 4, 15, 6, 8, 20653)

In [17]:
end_time = datetime.now()

In [18]:
end_time

datetime.datetime(2020, 7, 4, 15, 6, 8, 358750)

In [19]:
end_time - start_time

datetime.timedelta(microseconds=338097)

In [30]:
# tensor board 어디에 저장시킬 건지 지정
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [31]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True, # weight 시각화
    histogram_freq = 1,
)

In [39]:
# 실행 방법
%tensorboard --logdir /path/logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 11916.

## Training

http://localhost:6006

In [38]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Epoch 1/10
31/31 [==============================] - 14s 441ms/step - loss: 2.3271 - accuracy: 0.1074 - val_loss: 2.2985 - val_accuracy: 0.1169
Epoch 2/10
31/31 [==============================] - 14s 454ms/step - loss: 2.2812 - accuracy: 0.1085 - val_loss: 2.2752 - val_accuracy: 0.1421
Epoch 3/10
31/31 [==============================] - 20s 635ms/step - loss: 2.2493 - accuracy: 0.1539 - val_loss: 2.2067 - val_accuracy: 0.1744
Epoch 4/10
31/31 [==============================] - 15s 485ms/step - loss: 2.2132 - accuracy: 0.1901 - val_loss: 2.2107 - val_accuracy: 0.2087
Epoch 5/10
31/31 [==============================] - 14s 467ms/step - loss: 2.1535 - accuracy: 0.2273 - val_loss: 2.1293 - val_accuracy: 0.2268
Epoch 6/10
31/31 [==============================] - 11s 348ms/step - loss: 2.1223 - accuracy: 0.2035 - val_loss: 2.0122 - val_accuracy: 0.2752
Epoch 7/10
31/31 [==============================] - 10s 335ms/step - loss: 1.9966 - accuracy: 0.2645 - val_loss: 2.0060 - val_accuracy: 0.2601

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback
- tensorboard안에 customize한 그래프나 이미지 넣을 수 있도록

In [40]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    # matrix 그리는 함수
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [41]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    # 앞에서 그린 matrix를 image로 담음
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [42]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
# epoch이 시작됐을 때, 끝났을 때
# batch가 시작됐을 때, 끝났을 때
# 학습이 시작됐을 때, 끝났을 때
# 등등 callbacks를 언제 실행시킬 지 정해줌
# on_epoch_end=log_confusion_matrix: 함수 log_confusion_matrix를 epoch이 끝날 때마다 실행시키겠다

In [43]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
30/31 [============================>.] - ETA: 0s - loss: 1.8577 - accuracy: 0.2938

C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 13s 414ms/step - loss: 1.8636 - accuracy: 0.2934 - val_loss: 1.8490 - val_accuracy: 0.3125
Epoch 2/10
30/31 [============================>.] - ETA: 0s - loss: 1.8088 - accuracy: 0.3056

C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 13s 414ms/step - loss: 1.8063 - accuracy: 0.3048 - val_loss: 1.8529 - val_accuracy: 0.3508
Epoch 3/10
30/31 [============================>.] - ETA: 0s - loss: 1.8090 - accuracy: 0.3333

...

31/31 [==============================] - 15s 472ms/step - loss: 1.7193 - accuracy: 0.3619 - val_loss: 1.7891 - val_accuracy: 0.3558
Epoch 9/10
30/31 [============================>.] - ETA: 0s - loss: 1.6509 - accuracy: 0.4035

C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 16s 501ms/step - loss: 1.6459 - accuracy: 0.4047 - val_loss: 1.7843 - val_accuracy: 0.3669
Epoch 10/10
30/31 [============================>.] - ETA: 0s - loss: 1.6935 - accuracy: 0.3865

C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 15s 493ms/step - loss: 1.6892 - accuracy: 0.3881 - val_loss: 1.7926 - val_accuracy: 0.3599


## Expert
- 앞에선 tensorboard를 간단한게 fit에다가 callbacks를 여는 방법을 구현했음
- 이번엔 expert한 version에서 tensorboard 여는 방법 알아보자
- 장점 : 언제 넣을 것인지 시간을 정한다거나 등등 customize할 수 있음

In [44]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [45]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [46]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [47]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [48]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [49]:
file_write = tf.summary.create_file_writer(logdir)

In [50]:
for epoch in range(num_epochs):
    # Training
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)
    
    # 학습부분 끝나고나서
    with file_write.as_default():
        # training끝나고 마지막에 나왔던 image(시각화)를 tensorboard에 넣어주려고 함, 몇번째 step인지도 기록
        tf.summary.image('Input_image', images, step=step+(step*epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step+(step*epoch)) # loss나 accuracy같은 스칼라 값 넣어줌
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step+(step*epoch))
        
    # Evaluation
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

KeyboardInterrupt: 